In [1]:
import os
from copy import deepcopy
import json
# import xml.etree.ElementTree as ET
# from rich.tree import Tree
# from rich import print as rprint
import io
from typing import List, Union, Tuple, Dict
from collections.abc import Iterable
from tqdm import tqdm
# import pdfplumber
# import fitz 
import numpy as np
import pandas as pd
import requests
# import xmltodict
import re
from pypdf import PdfReader, PdfWriter
from difflib import SequenceMatcher
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.document import DocumentStream
from docling.pipeline.vlm_pipeline import VlmPipeline
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, BitsAndBytesConfig, LogitsProcessor, LogitsProcessorList
# device = torch.device("mps")
from table_link_to_excel import _curl_get_text, _extract_pmc_info, _try_pmc_direct_table_download, _flatten_columns, sanitize_sheet_name, fetch_html, fetch_pmc_fulltext_xml, pick_table, table_to_dataframe, _clean_text
from bs4 import BeautifulSoup
from gwas_column_matching_engine import GWASColumnMatchingEngine
from gwas_table_extraction import *

/Users/justpqa/advpai/venv/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/Users/justpqa/advpai/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Try to look at the pdf

Try to look at the website of the paper instead

Try to use Docling

In [ ]:
# first we need to know the number of col of a table to make sure we try the right orientation with docling
def extract_tables_num_col_lst_from_pmc(pmcid):
    url = f"https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/{pmcid}/unicode"

    response = requests.get(url)
    tables_num_col_lst = []
    if response.status_code == 200:
        data = response.json()
        for d in data:
            doc = d["documents"]
            for p in doc:
                passage = p["passages"]
                for item in passage:
                    if item.get("infons", "").get("type", "").lower() == "table" and "text" in item:
                        table_str = item["text"]
                        num_col = 0
                        for row in table_str.split("\t \t"):
                            row_lst = row.split("\t")
                            num_col = max(num_col, len(row_lst))
                        tables_num_col_lst.append(num_col)
        print(f"Successfully retrieve number of columns")
    else:
        print(f"Failed to retrieve number of columns: {response.status_code}")
        
    return tables_num_col_lst

In [ ]:
# Cleaning value with tag
def clean_cell(val):
    tag_pattern = r'\s[a-z]$'
    if isinstance(val, str):
        return re.sub(tag_pattern, '', val)
    return val

def clean_headers(df):
    tag_pattern = r'\s[a-z]$'
    new_cols = []
    seen = {}
    for col in df.columns:
        if pd.isna(col):
            new_cols.append("")
        else:
            # 1. Apply the regex to the name string
            clean_name = re.sub(tag_pattern, '', str(col))
            # 2. Handle duplicates (e.g., if 'Price a' and 'Price b' both become 'Price')
            if clean_name in seen:
                seen[clean_name] += 1
                clean_name = f"{clean_name}_{seen[clean_name]}"
            else:
                seen[clean_name] = 0
            new_cols.append(clean_name)
    df.columns = new_cols
    return df

In [ ]:
# detect specific stuff like snp and pvalue
def contains_valid_snp(row):
    row_snp_pattern = r"\b(?:rs|s)\d+\S*"
    return any(re.search(row_snp_pattern, str(value)) for value in row)

def contains_valid_pvalue(row):
    row_pvalue_pattern = r"\d+\.\d+"
    return any(re.search(row_pvalue_pattern, str(value)) for value in row)

In [ ]:
# extract table list
def extract_tables_lst_from_pdf_and_num_col(file_name, tables_num_col_lst):
    reader = PdfReader(file_name)
    options = PdfPipelineOptions()
    options.table_structure_options.mode = TableFormerMode.ACCURATE
    converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=options)
        }
    )
    df_lst = []
    row_snp_pattern = r"\b(?:rs|s)\d+\S*"
    row_pvalue_pattern = r"\d+\.\d+"

    page_num = 1
    while len(df_lst) < len(tables_num_col_lst) and page_num <= len(reader.pages):
        for angle in [0, 90]: # Try normal, then try rotated
            writer = PdfWriter()
            page = reader.pages[page_num - 1]
            
            if angle != 0:
                page.rotate(angle)
            
            writer.add_page(page)
            
            # Convert just this one page
            pdf_buffer = io.BytesIO()
            writer.write(pdf_buffer)
            pdf_buffer.seek(0)
            
            doc_stream = DocumentStream(name=f"page_{page_num}.pdf", stream=pdf_buffer)
            result = converter.convert(doc_stream)
            
            # Check if this rotation produced valid table rows
            temp_dfs = []
            for table in result.document.tables:
                df = table.export_to_dataframe()
                # check if table is empty
                if (not df.empty):
                    # Case 1: continue from previous table
                    if len(df_lst) > 0 and df.shape[1] == df_lst[-1].shape[1]:
                        # extra filters for tables that are snp related, we need to remove rows that do not have snp id
                        # often are separation between sections
                        for col in df.columns:
                            # first modify "" -> nan
                            df[col] = df[col].replace(r'^\s*$', np.nan, regex=True).ffill()
                        df["valid_row"] = df.apply(lambda x: contains_valid_snp(x) and contains_valid_pvalue(x), axis=1)
                        df = df[df["valid_row"]].drop("valid_row", axis=1).reset_index().drop("index", axis = 1)
                        # some cell have the tag the end (often include a space and a small letter)
                        df = df.map(clean_cell) 
                        df = clean_headers(df) 
                        if df_lst[-1].columns.equals(df.columns):
                            df_lst[-1] = pd.concat([df_lst[-1], df], ignore_index = True)
                        elif df.shape[1] == tables_num_col_lst[len(df_lst) + len(temp_dfs)]:
                            # fail that test => add to temp since this is a new table
                            temp_dfs.append(df)
                    # Case 2: new table
                    elif (len(df_lst) + len(temp_dfs)) < len(tables_num_col_lst) and df.shape[1] == tables_num_col_lst[len(df_lst) + len(temp_dfs)]: 
                        # extra filters for tables that are snp related, we need to remove rows that do not have snp id
                        # often are separation between sections
                        for col in df.columns:
                            # first modify "" -> nan
                            df[col] = df[col].replace(r'^\s*$', np.nan, regex=True).ffill()
                        df["valid_row"] = df.apply(lambda x: contains_valid_snp(x) and contains_valid_pvalue(x), axis=1)
                        df = df[df["valid_row"]].drop("valid_row", axis=1).reset_index().drop("index", axis = 1)
                        # some cell have the tag the end (often include a space and a small letter)
                        df = df.map(clean_cell) 
                        df = clean_headers(df)             
                        temp_dfs.append(df)
            if temp_dfs:
                df_lst.extend(temp_dfs)
                break
            
        page_num += 1

    return df_lst

In [ ]:
def extract_tables_lst_from_paper(pmcid, file_name, table_inx_to_extract=[]):
    tables_num_col_lst = extract_tables_num_col_lst_from_pmc(pmcid)
    if len(table_inx_to_extract) > 0:
        tables_num_col_lst = [tables_num_col_lst[i] for i in table_inx_to_extract]
    df_lst = extract_tables_lst_from_pdf_and_num_col(file_name, tables_num_col_lst)
    return df_lst

In [ ]:
pmcid = "PMC10497850"
file_name = "papers/ACEL-22-e13938.pdf"
df_lst = extract_tables_lst_from_paper(pmcid, file_name)
for i in [3, 4]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

In [ ]:
pmcid = "PMC10115645"
file_name = "papers/41591_2023_Article_2268.pdf"  # Can be a local path or a URL
df_lst = extract_tables_lst_from_paper(pmcid, file_name)
for i in [1, 2]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

In [ ]:
pmcid = "PMC9622429"
file_name = "papers/nihms-1797266.pdf"  # Can be a local path or a URL
df_lst = extract_tables_lst_from_paper(pmcid, file_name)  
for i in [3, 5, 6, 7, 8]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

In [ ]:
pmcid = "PMC10615750"
file_name = "papers/Recent paper on AD GWAS (1).pdf"  # Can be a local path or a URL
df_lst = extract_tables_lst_from_paper(pmcid, file_name)  
for i in [1, 2]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

In [ ]:
pmcid = "PMC6677735"
file_name = "papers/s42003-019-0537-9.pdf"  # Can be a local path or a URL
df_lst = extract_tables_lst_from_paper(pmcid, file_name)  
for i in [1]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

In [ ]:
pmcid = "PMC10286470"
file_name = "papers/s13024-023-00633-4.pdf"  # Can be a local path or a URL
df_lst = extract_tables_lst_from_paper(pmcid, file_name)  
for i in [2, 3]:
    df_lst[i-1].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)

In [ ]:
# pmcid = "PMC9945061"
# file_name = "PMID36809323_table_3.pdf"
# df_lst = extract_tables_lst_from_paper(pmcid, file_name)  
# for i in [3]:
#     df_lst[i-3].to_csv(f"tables/{file_name.split('/')[-1].replace('.pdf', '')}_table_{i}.csv", index=False)
# problem in PMC api

Try to map columns to reference columns

In [ ]:
# Now try to map the columns with the actual col in advp
# referencing_cols = pd.read_csv("Rules for harmonizing ADVP papers - Main cols.csv")
# referencing_cols["column_with_context"] = referencing_cols.apply(lambda x: x["column"] if pd.isna(x["description"]) else x["column"] + ": " + x["description"], axis = 1)
# # referencing_cols

In [ ]:
# # NOTE: before working in the main cell for mapping columns, write inference code here + setting up models here
# def create_embeddings_from_model(sentences, model, tokenizer):
#     input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

#     # get token embeddings
#     with torch.no_grad():
#         output = model(**input)
#     token_embeddings = output[0]

#     # extract mask and mean pooling for sentence embeddings
#     input_mask_expanded = input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
#     sentence_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

#     # final normalization
#     sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

#     return sentence_embeddings

# # setting up models
# embeddings_model_name = "NeuML/pubmedbert-base-embeddings"
# embeddings_model_tokenizer = AutoTokenizer.from_pretrained(embeddings_model_name)
# embeddings_model = AutoModel.from_pretrained(embeddings_model_name)
# embeddings_model.eval()

In [ ]:
# class SingleTokenBiasProcessor(LogitsProcessor):
#     def __init__(self, token_ids, bias_value):
#         self.token_ids = token_ids
#         self.bias_value = bias_value

#     def __call__(self, input_ids, scores):
#         # Create a mask for allowed tokens
#         mask = torch.full_like(scores, -float("inf"))
#         for tid in self.token_ids:
#             mask[:, tid] = self.bias_value
#         return scores + mask

# def reranking_with_llm(col, candidates, llm_model, llm_model_tokenizer):
#     """
#     LLM acts as a re-ranker to pick the best match from a list of candidates.
#     """
#     # Format the candidates as a numbered list for the LLM

#     prompt = f"""Task: Map clinical table headers to GWAS standard ontology, return a single number for the best choice

# Header: "p-value: 0.001, 5e-8, 0.43"
# Candidates: 
#     1. P-value: The statistical significance of the association. Keywords: P, P-value, P_adj, FDR. Examples: 5.0E-08, 0.0012, 1.2 x 10^-5, 0.05.
#     2. Effect Size: The magnitude and direction of the association. Keywords: Beta, OR, HR, Estimate. Examples: Beta=0.25, OR=1.45, HR=1.12, Log(OR)=0.37.
#     3. SNP: Variant identifier, or snp idenifier, or chr:pos. Keywords: chr:position, chr:pos, Variant, rsID, RS number, MarkerName, rs. Examples: rs12345, 20:45269867, 19:45411941:T:C, chr19:45411941, rs429358 (APOE ε4).
# Best Match: 1

# Header: "rs_number: rs123, rs456, rs789"
# Candidates: 
#     1. Chr: Genomic chromosome identifier. Keywords: CHR, Chrom, Chromosome. Examples: 1, 19, X, chr19, chrX.
#     2. Position: Genomic coordinate location. Keywords: BP, POS, Base Pair, start, end. Examples: 45411941, 10240500:10248600 (range), build 37.
#     3. SNP: Variant identifier, or snp idenifier, or chr:pos. Keywords: chr:position, chr:pos, Variant, rsID, RS number, MarkerName, rs. Examples: rs12345, 20:45269867, 19:45411941:T:C, chr19:45411941, rs429358 (APOE ε4).
# Best Match: 2

# Header: "{col}"
# Candidates: 
#     - {candidates[0]}
#     - {candidates[1]}
#     - {candidates[2]}
# Best Match: """

#     allowed_indices = [str(i+1) for i in range(len(candidates))]
#     allowed_token_ids = [llm_model_tokenizer.encode(idx, add_special_tokens=False)[0] for idx in allowed_indices]
    
#     # logit bias to limit tokens that can be output
#     bias_processor = SingleTokenBiasProcessor(allowed_token_ids, 100.0)
#     logits_processor = LogitsProcessorList([bias_processor])

#     # 4. Generate exactly ONE token
#     inputs = llm_model_tokenizer(prompt, return_tensors="pt").to(device)
    
#     with torch.no_grad():
#         output = llm_model.generate(
#             **inputs,
#             max_new_tokens=1,      # Force exactly one token
#             logits_processor=logits_processor, # Force it to be one of our numbers
#             pad_token_id=llm_model_tokenizer.eos_token_id,
#             do_sample=False        # Greedy decoding for consistency
#         )

#     # 5. Extract and Convert to Integer
#     new_token = output[0][-1]
#     predicted_text = llm_model_tokenizer.decode(new_token).strip()
    
#     try:
#         idx = int(predicted_text) - 1 # Convert back to 0-based list index
#         return candidates[idx]
#     except (ValueError, IndexError):
#         return col

# llm_model_name = "stanford-crfm/BioMedLM"
# llm_model_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
# llm_model = AutoModelForCausalLM.from_pretrained(
#     llm_model_name,
#     torch_dtype=torch.bfloat16
# ).to(device)
# llm_model.eval()

In [ ]:
# # Try to first convert any abbreviation, then we match column with right semantic
# gwas_abbreviation_dict = {
#     "CHR": "Chromosome number",
#     "BP": "Base-pair position",
#     "POS": "Position",
#     "SNP": "Single nucleotide polymorphism identifier",
#     "RS": "Reference Single nucleotide polymorphism",
#     "VAR": "Variant",
#     "ID": "identifier",
#     "A1": "Effect allele / tested allele",
#     "A2": "Other allele / non-effect allele",
#     "REF": "Reference allele (genome reference)",
#     "ALT": "Alternate allele",
#     "EA": "Effect Allele",
#     "NEA": "Non-Effect Allele",
#     "RA": "Risk Allele",
#     "OA": "Other Allele",
#     "AF": "Allele Frequency (general term)",
#     "RAF": "Risk Allele Frequency",
#     "EAF": "Effect Allele Frequency",
#     "MAF": "Minor Allele Frequency",
#     "BETA": "Effect size (regression coefficient)",
#     "OR": "Odds Ratio",
#     "SE": "Standard Error of effect estimate",
#     "Z": "Z-score statistic",
#     "T": "T-statistic",
#     "CI": "Confidence Interval",
#     "P": "P-value",
#     "PVAL": "P-value",
#     "LOGP": "Negative log10 P-value",
#     "Q": "Heterogeneity statistic (meta-analysis)",
#     "I2": "I-squared heterogeneity metric",
#     "HWE": "Hardy-Weinberg Equilibrium test statistic",
#     "INFO": "Imputation quality score",
#     "R2": "Imputation accuracy metric",
#     "CALLRATE": "Genotype call rate",
#     "MISSING": "Missing genotype rate",
#     "N": "Total sample size",
#     "N_CASES": "Number of cases (for binary traits)",
#     "N_CONTROLS": "Number of controls (for binary traits)",
#     "EUR": "European ancestry",
#     "AFR": "African ancestry",
#     "ASN": "Asian ancestry",
#     "AMR": "Admixed American ancestry",
#     "SAS": "South Asian ancestry",
#     "EAS": "East Asian ancestry",
#     "LD": "Linkage Disequilibrium",
#     "DPRIME": "LD D’ value",
#     "CADD": "CADD score (functional impact)",
#     "EQTL": "Expression quantitative trait locus",
#     "PQTL": "Protein QTL",
#     "GWGAS": "Gene-wide association study",
#     "PRS": "Polygenic Risk Score",
#     "PGS": "Polygenic Score",
#     "QC": "Quality Control",
#     "MA": "Meta-analysis",
#     "HLA": "Human Leukocyte Antigen region",
#     "HR": "Hazard ratio",
#     "HET": "Heterogeneity test",
#     "APOE4": "APOE ε4",
#     "APOE*4": "APOE ε4",
#     "#": "Number of",
#     "frq": "Frequency",
#     'β': "Effect",
#     "nsnps": "Number of Variants"
# }

In [ ]:
# def clean_col(col):
#     """
#     Clean a column by replacing any possible abbreviation with their actual meaning for better semantic matching
#     """
#     new_col = col
#     for abb in gwas_abbreviation_dict:
#         if re.search(fr"[^a-zA-Z]{abb.lower()}[^a-zA-Z]", new_col.lower()):
#             new_col = re.sub(fr"([^a-zA-Z]){abb.lower()}([^a-zA-Z])", fr"\1{gwas_abbreviation_dict[abb]}\2", new_col.lower())
#         elif re.search(fr"^{abb.lower()}[^a-zA-Z]", new_col.lower()):
#             new_col = re.sub(fr"^{abb.lower()}([^a-zA-Z])", fr"{gwas_abbreviation_dict[abb]}\1", new_col.lower())
#         elif re.search(fr"[^a-zA-Z]{abb.lower()}$", new_col.lower()):
#             new_col = re.sub(fr"([^a-zA-Z]){abb.lower()}$", fr"\1{gwas_abbreviation_dict[abb]}", new_col.lower())
#         elif re.search(fr"^{abb.lower()}$", new_col.lower()):
#             new_col = re.sub(fr"{abb.lower()}", gwas_abbreviation_dict[abb], new_col.lower())
#     # new_col = new_col.replace(".", " ")
#     return new_col

In [ ]:
# def make_col_prompt(col, example_values, num_example_values = 5):
#     """
#     Based on the column title and some possible values of that columns, try to make a prompt
#     """
#     col_prompt = f"{col}: "
#     for i in range(min(num_example_values, len(example_values))):
#         col_prompt += f"{example_values[i]}, "
#     return col_prompt

In [ ]:
# def match_single_col_to_ref_col(col, ref_col_lst, ref_col_embeddings, embeddings_model, embeddings_model_tokenizer):
#     """
#     match a single column to reference col given column, the list of referencing col and their embeddings
#     """
#     # calculate column embedding
#     # col_embeddings = embeddings_model.encode(col, normalize_embeddings=True)
#     # multi_index_pattern = r".+\..+"
#     # if re.search(multi_index_pattern, col):
#     #     col = col.replace(".", " ")
#     #     # sub_col_lst = col.split(".")
#     #     # sub_col_embeddings = create_embeddings_from_model(sub_col_lst, embeddings_model, embeddings_model_tokenizer)
#     #     # col_embeddings = torch.mean(sub_col_embeddings, dim = 0)
#     #     col = col.replace(".", " ")
#     #     col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
#     # else:
#     #     col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
#     col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
#     col_embeddings = col_embeddings.reshape(-1, 1)

#     # calculate similarity score
#     scores = torch.matmul(ref_col_embeddings, col_embeddings).reshape(-1)

#     # sort similairty score
#     top_k_indices = torch.argsort(scores, descending=True)

#     # verify if we even got good enough similarity
#     best_inx = top_k_indices[0].item()
#     best_score = scores[best_inx].item()
#     # second_best_inx = top_k_indices[1].item()
#     # second_best_score = scores[second_best_inx].item()
#     # need a threshold for score or else, just return col
#     # if best_score < 0.4: 
#     #     return (col, 1)
    
#     # # now do rerank
#     # candidates = [ref_col_lst[inx] for inx in top_k_indices if scores[inx] >= 0.4]
#     # candidates_scores = [scores[inx] for inx in top_k_indices if scores[inx] >= 0.4]
#     # best_ref_col, best_ref_col_scores = reranking_from_model(col, candidates, candidates_scores)
#     # return (best_ref_col, best_ref_col_scores)

#     if best_score >= 0.4:
#         return (ref_col_lst[best_inx], best_score)
#     return (col, 1)

# def match_many_col_to_ref_col(df, ref_col_df, embeddings_model, embeddings_model_tokenizer):
#     """
#     Given a list of column and a dataframe (could be dict later on if that fits better),
#     try to match each column to the best fitted reference col, 
#     return a dict of ref col : list of (col, cleaned col prompt, score)
#     """
#     # prepare the embeddings for reference col since we reuse them
#     ref_col_lst = ref_col_df["column"].to_list()
#     ref_col_context_lst = ref_col_df["column_with_context"].to_list()
#     ref_col_embeddings = create_embeddings_from_model(ref_col_context_lst, embeddings_model, embeddings_model_tokenizer)

#     # conduct matching
#     multi_index_pattern = r"^.+\..+$" # need multi index pattern for handling multi index
#     ref_col_to_col_lst = {}
#     for col in df.columns:

#         # extract values needed for prompt
#         cleaned_col = clean_col(col)
#         example_values = df[col].unique().tolist()
#         # prompt: {col}: example, need to delete all : first

#         if re.search(multi_index_pattern, cleaned_col.strip()):
#             # try to assess each part and see if which one have highest score
#             best_ref_col, best_score = None, 0
#             best_cleaned_sub_col_prompt = None
#             for sub_col in cleaned_col.split("."):

#                 # make column prompt for each sub col
#                 cleaned_sub_col_prompt = make_col_prompt(sub_col, example_values)
                
#                 # matching and compare
#                 ref_col, score = match_single_col_to_ref_col(cleaned_sub_col_prompt, ref_col_lst, ref_col_embeddings, embeddings_model, embeddings_model_tokenizer)
#                 if score > best_score and ref_col != cleaned_sub_col_prompt:
#                     best_ref_col = ref_col
#                     best_score = score
#                     best_cleaned_sub_col_prompt = cleaned_sub_col_prompt
            
#             # if we have a best one vs not
#             if best_ref_col is not None:
#                 if best_ref_col not in ref_col_to_col_lst:
#                     ref_col_to_col_lst[best_ref_col] = []
#                 ref_col_to_col_lst[best_ref_col].append((col, best_cleaned_sub_col_prompt, score))
#             else:
#                 if col not in ref_col_to_col_lst:
#                     ref_col_to_col_lst[col] = []
#                 ref_col_to_col_lst[col].append((col, best_cleaned_sub_col_prompt, 1))
#         else:
#             # match a single col with best fit referencing col and return a dict
#             # make the col prompt
#             # extra steps to remove .
#             cleaned_col = cleaned_col.replace(".", "")
#             cleaned_col_prompt = make_col_prompt(cleaned_col, example_values)

#             # matching for best col
#             ref_col, score = match_single_col_to_ref_col(cleaned_col_prompt, ref_col_lst, ref_col_embeddings, embeddings_model, embeddings_model_tokenizer)
#             # if we still get same col
#             if ref_col == cleaned_col_prompt: 
#                 ref_col = col
#             if ref_col not in ref_col_to_col_lst:
#                 ref_col_to_col_lst[ref_col] = []
#             ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))
#         # ref_col, score = match_single_col_to_ref_col(cleaned_col_prompt, ref_col_lst, ref_col_embeddings)
#         # if ref_col == cleaned_col_prompt: 
#         #     ref_col = col
#         # if ref_col not in ref_col_to_col_lst:
#         #     ref_col_to_col_lst[ref_col] = []
#         # ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))

#     return ref_col_to_col_lst

In [ ]:
# def match_single_col_to_ref_col_with_llm(col, ref_col_lst, ref_col_embeddings):
#     """
#     match a single column to reference col given column, the list of referencing col and their embeddings
#     """
#     calculate column embedding
#     col_embeddings = embeddings_model.encode(col, normalize_embeddings=True)
#     multi_index_pattern = r".+\..+"
#     if re.search(multi_index_pattern, col):
#         col = col.replace(".", " ")
#         # sub_col_lst = col.split(".")
#         # sub_col_embeddings = create_embeddings_from_model(sub_col_lst, embeddings_model, embeddings_model_tokenizer)
#         # col_embeddings = torch.mean(sub_col_embeddings, dim = 0)
#         col = col.replace(".", " ")
#         col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
#     else:
#         col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
#     col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
#     col_embeddings = col_embeddings.reshape(-1, 1)

#     calculate similarity score
#     scores = torch.matmul(ref_col_embeddings, col_embeddings).reshape(-1)

#     sort similairty score
#     top_k_indices = torch.argsort(scores, descending=True)

#     verify if we even got good enough similarity
#     best_inx = top_k_indices[0].item()
#     best_score = scores[best_inx].item()
#     second_best_inx = top_k_indices[1].item()
#     second_best_score = scores[second_best_inx].item()
#     need a threshold for score or else, just return col
#     if best_score < 0.4: 
#         return (col, 1)
    
#     # now do rerank
#     candidates = [ref_col_lst[inx] for inx in top_k_indices if scores[inx] >= 0.4]
#     candidates_scores = [scores[inx] for inx in top_k_indices if scores[inx] >= 0.4]
#     best_ref_col, best_ref_col_scores = reranking_from_model(col, candidates, candidates_scores)
#     return (best_ref_col, best_ref_col_scores)

#     if best_score < 0.4:
#         return col
#     else:
#         extract top 3 candidates
#         candidates = []
#         for i in range(3):
#             inx = top_k_indices[i]
#             candidates.append(ref_col_lst[inx])
#         best_col = reranking_with_llm(col, candidates, llm_model, llm_model_tokenizer)
#         return best_col

# def match_many_col_to_ref_col_with_llm(df, ref_col_df):
#     """
#     Given a list of column and a dataframe (could be dict later on if that fits better),
#     try to match each column to the best fitted reference col, 
#     return a dict of ref col : list of (col, cleaned col prompt, score)
#     """
#     prepare the embeddings for reference col since we reuse them
#     ref_col_lst = ref_col_df["column"].to_list()
#     ref_col_context_lst = ref_col_df["column_with_context"].to_list()
#     ref_col_embeddings = create_embeddings_from_model(ref_col_context_lst, embeddings_model, embeddings_model_tokenizer)

#     conduct matching
#     multi_index_pattern = r"^.+\..+$" # need multi index pattern for handling multi index
#     ref_col_to_col_lst = {}
#     for col in df.columns:

#         extract values needed for prompt
#         cleaned_col = clean_col(col)
#         example_values = df[col].unique().tolist()
#         prompt: {col}: example, need to delete all : first

#         if re.search(multi_index_pattern, cleaned_col.strip()):
#             try to assess each part and see if which one have highest score
#             best_ref_col, best_score = None, 0
#             best_cleaned_sub_col_prompt = None
#             for sub_col in cleaned_col.split("."):

#                 make column prompt for each sub col
#                 cleaned_sub_col_prompt = make_col_prompt(sub_col, example_values)
                
#                 matching and compare
#                 ref_col, score = match_single_col_to_ref_col(cleaned_sub_col_prompt, ref_col_lst, ref_col_embeddings)
#                 if score > best_score and ref_col != cleaned_sub_col_prompt:
#                     best_ref_col = ref_col
#                     best_score = score
#                     best_cleaned_sub_col_prompt = cleaned_sub_col_prompt
            
#             if we have a best one vs not
#             if best_ref_col is not None:
#                 if best_ref_col not in ref_col_to_col_lst:
#                     ref_col_to_col_lst[best_ref_col] = []
#                 ref_col_to_col_lst[best_ref_col].append((col, best_cleaned_sub_col_prompt, score))
#             else:
#                 if col not in ref_col_to_col_lst:
#                     ref_col_to_col_lst[col] = []
#                 ref_col_to_col_lst[col].append((col, best_cleaned_sub_col_prompt, 1))
#         else:
#             match a single col with best fit referencing col and return a dict
#             make the col prompt
#             extra steps to remove .
#             cleaned_col = cleaned_col.replace(".", "")
#             cleaned_col_prompt = make_col_prompt(cleaned_col, example_values)

#             matching for best col
#             ref_col, score = match_single_col_to_ref_col(cleaned_col_prompt, ref_col_lst, ref_col_embeddings)
#             if we still get same col
#             if ref_col == cleaned_col_prompt: 
#                 ref_col = col
#             if ref_col not in ref_col_to_col_lst:
#                 ref_col_to_col_lst[ref_col] = []
#             ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))
#         ref_col, score = match_single_col_to_ref_col(cleaned_col_prompt, ref_col_lst, ref_col_embeddings)
#         if ref_col == cleaned_col_prompt: 
#             ref_col = col
#         if ref_col not in ref_col_to_col_lst:
#             ref_col_to_col_lst[ref_col] = []
#         ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))

#     return ref_col_to_col_lst

# def match_many_col_to_ref_col(df, ref_col_df, embeddings_model, embeddings_model_tokenizer):
#     """
#     Given a list of column and a dataframe (could be dict later on if that fits better),
#     try to match each column to the best fitted reference col, 
#     return a dict of ref col : list of (col, cleaned col prompt, score)
#     """
#     prepare the embeddings for reference col since we reuse them
#     ref_col_lst = ref_col_df["column"].to_list()
#     ref_col_context_lst = ref_col_df["column_with_context"].to_list()
#     ref_col_embeddings = create_embeddings_from_model(ref_col_context_lst, embeddings_model, embeddings_model_tokenizer)

#     conduct matching
#     multi_index_pattern = r"^.+\..+$" # need multi index pattern for handling multi index
#     ref_col_to_col_lst = {}
#     for col in df.columns:

#         extract values needed for prompt
#         cleaned_col = clean_col(col)
#         example_values = df[col].unique().tolist()
#         prompt: {col}: example, need to delete all : first

#         if re.search(multi_index_pattern, cleaned_col.strip()):
#             try to assess each part and see if which one have highest score
#             best_ref_col, best_score = None, 0
#             best_cleaned_sub_col_prompt = None
#             for sub_col in cleaned_col.split("."):

#                 make column prompt for each sub col
#                 cleaned_sub_col_prompt = make_col_prompt(sub_col, example_values)
                
#                 matching and compare
#                 ref_col, score = match_single_col_to_ref_col(cleaned_sub_col_prompt, ref_col_lst, ref_col_embeddings, embeddings_model, embeddings_model_tokenizer)
#                 if score > best_score and ref_col != cleaned_sub_col_prompt:
#                     best_ref_col = ref_col
#                     best_score = score
#                     best_cleaned_sub_col_prompt = cleaned_sub_col_prompt
            
#             if we have a best one vs not
#             if best_ref_col is not None:
#                 if best_ref_col not in ref_col_to_col_lst:
#                     ref_col_to_col_lst[best_ref_col] = []
#                 ref_col_to_col_lst[best_ref_col].append((col, best_cleaned_sub_col_prompt, score))
#             else:
#                 if col not in ref_col_to_col_lst:
#                     ref_col_to_col_lst[col] = []
#                 ref_col_to_col_lst[col].append((col, best_cleaned_sub_col_prompt, 1))
#         else:
#             match a single col with best fit referencing col and return a dict
#             make the col prompt
#             extra steps to remove .
#             cleaned_col = cleaned_col.replace(".", "")
#             cleaned_col_prompt = make_col_prompt(cleaned_col, example_values)

#             matching for best col
#             ref_col, score = match_single_col_to_ref_col(cleaned_col_prompt, ref_col_lst, ref_col_embeddings, embeddings_model, embeddings_model_tokenizer)
#             if we still get same col
#             if ref_col == cleaned_col_prompt: 
#                 ref_col = col
#             if ref_col not in ref_col_to_col_lst:
#                 ref_col_to_col_lst[ref_col] = []
#             ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))
#         ref_col, score = match_single_col_to_ref_col(cleaned_col_prompt, ref_col_lst, ref_col_embeddings)
#         if ref_col == cleaned_col_prompt: 
#             ref_col = col
#         if ref_col not in ref_col_to_col_lst:
#             ref_col_to_col_lst[ref_col] = []
#         ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))

#     return ref_col_to_col_lst

In [ ]:
# for file in os.listdir("./tables"):
#     if "table" in file and ".csv" in file and "harmonized" not in file:
#         print(file)
#         df = pd.read_csv(f"./tables/{file}")
#         df.columns = ['' if 'Unnamed:' in col else col for col in df.columns]
#         col_to_ref_col = match_many_col_to_ref_col(df, referencing_cols)
#         for ref_col in col_to_ref_col:
#             print(f"{ref_col}: {col_to_ref_col[ref_col]}")
#         print()

Rewrite the matching engine as an object

In [ ]:
# Now try to map the columns with the actual col in advp
referencing_cols = pd.read_csv("Rules for harmonizing ADVP papers - Main cols.csv")
referencing_cols

In [ ]:
class SingleTokenBiasProcessor(LogitsProcessor):
    def __init__(self, token_ids, bias_value):
        self.token_ids = token_ids
        self.bias_value = bias_value

    def __call__(self, input_ids, scores):
        # Create a mask for allowed tokens
        mask = torch.full_like(scores, -float("inf"))
        for tid in self.token_ids:
            mask[:, tid] = self.bias_value
        return scores + mask

In [ ]:
class GWASColumnMatchingEngine:
    def __init__(self, referencing_col_df: pd.DataFrame, embeddings_model_name: str = "NeuML/pubmedbert-base-embeddings", 
                 use_llm: bool = False, llm_model_name: str = "stanford-crfm/BioMedLM", device: str = "cpu"):
        # df of referencing col
        if not ("column" in referencing_col_df.columns and "description" in referencing_col_df.columns):
            raise Exception("Error: Dataframe for referencing columns need to have 2 columns: column and description")
        self.referencing_col_lst = referencing_col_df["column"].to_list()
        self.referencing_col_context_lst = referencing_col_df.apply(lambda x: x["column"] if pd.isna(x["description"]) else x["column"] + ": " + x["description"], axis = 1).to_list()
        if use_llm:
            self.referencing_col_to_col_context = {c: cc for c, cc in zip(self.referencing_col_lst, self.referencing_col_context_lst)}
            self.referencing_col_context_to_col = {cc: c for c, cc in zip(self.referencing_col_lst, self.referencing_col_context_lst)}

        # Try to first convert any abbreviation, then we match column with right semantic
        self.gwas_abbreviation_dict = {
            "CHR": "Chromosome number",
            "BP": "Base-pair position",
            "POS": "Position",
            "SNP": "Single nucleotide polymorphism identifier",
            "RS": "Reference Single nucleotide polymorphism",
            "VAR": "Variant",
            "ID": "identifier",
            "A1": "Effect allele / tested allele",
            "A2": "Other allele / non-effect allele",
            "REF": "Reference allele (genome reference)",
            "ALT": "Alternate allele",
            "EA": "Effect Allele",
            "NEA": "Non-Effect Allele",
            "RA": "Risk Allele",
            "OA": "Other Allele",
            "AF": "Allele Frequency (general term)",
            "RAF": "Risk Allele Frequency",
            "EAF": "Effect Allele Frequency",
            "MAF": "Minor Allele Frequency",
            "BETA": "Effect size (regression coefficient)",
            "OR": "Odds Ratio",
            "SE": "Standard Error of effect estimate",
            "Z": "Z-score statistic",
            "T": "T-statistic",
            "CI": "Confidence Interval",
            "P": "P-value",
            "PVAL": "P-value",
            "LOGP": "Negative log10 P-value",
            "Q": "Heterogeneity statistic (meta-analysis)",
            "I2": "I-squared heterogeneity metric",
            "HWE": "Hardy-Weinberg Equilibrium test statistic",
            "INFO": "Imputation quality score",
            "R2": "Imputation accuracy metric",
            "CALLRATE": "Genotype call rate",
            "MISSING": "Missing genotype rate",
            "N": "Total sample size",
            "N_CASES": "Number of cases (for binary traits)",
            "N_CONTROLS": "Number of controls (for binary traits)",
            "EUR": "European ancestry",
            "AFR": "African ancestry",
            "ASN": "Asian ancestry",
            "AMR": "Admixed American ancestry",
            "SAS": "South Asian ancestry",
            "EAS": "East Asian ancestry",
            "LD": "Linkage Disequilibrium",
            "DPRIME": "LD D’ value",
            "CADD": "CADD score (functional impact)",
            "EQTL": "Expression quantitative trait locus",
            "PQTL": "Protein QTL",
            "GWGAS": "Gene-wide association study",
            "PRS": "Polygenic Risk Score",
            "PGS": "Polygenic Score",
            "QC": "Quality Control",
            "MA": "Meta-analysis",
            "HLA": "Human Leukocyte Antigen region",
            "HR": "Hazard ratio",
            "HET": "Heterogeneity test",
            "APOE4": "APOE ε4",
            "APOE*4": "APOE ε4",
            "#": "Number of",
            "frq": "Frequency",
            'β': "Effect",
            "nsnps": "Number of Variants"
        }

        # embeddings model
        self.embeddings_model_tokenizer = AutoTokenizer.from_pretrained(embeddings_model_name)
        self.embeddings_model = AutoModel.from_pretrained(embeddings_model_name)
        self.embeddings_model.eval()

        # also make col embeddings
        self.referencing_col_embeddings = self.create_col_embeddings_from_model(self.referencing_col_context_lst)

        # llm model
        self.use_llm = use_llm
        if use_llm:
            self.llm_model_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
            self.llm_model = AutoModelForCausalLM.from_pretrained(
                llm_model_name,
                dtype=torch.bfloat16
            ).to(device)
            self.llm_model.eval()

        # device
        self.device = device

    def clean_col(self, col: str) -> str:
        """
        Clean a column by replacing any possible abbreviation with their actual meaning for better semantic matching
        """
        new_col = col
        for abb in self.gwas_abbreviation_dict:
            if re.search(fr"[^a-zA-Z]{abb.lower()}[^a-zA-Z]", new_col.lower()):
                new_col = re.sub(fr"([^a-zA-Z]){abb.lower()}([^a-zA-Z])", fr"\1{self.gwas_abbreviation_dict[abb]}\2", new_col.lower())
            elif re.search(fr"^{abb.lower()}[^a-zA-Z]", new_col.lower()):
                new_col = re.sub(fr"^{abb.lower()}([^a-zA-Z])", fr"{self.gwas_abbreviation_dict[abb]}\1", new_col.lower())
            elif re.search(fr"[^a-zA-Z]{abb.lower()}$", new_col.lower()):
                new_col = re.sub(fr"([^a-zA-Z]){abb.lower()}$", fr"\1{self.gwas_abbreviation_dict[abb]}", new_col.lower())
            elif re.search(fr"^{abb.lower()}$", new_col.lower()):
                new_col = re.sub(fr"{abb.lower()}", self.gwas_abbreviation_dict[abb], new_col.lower())
        # new_col = new_col.replace(".", " ")
        return new_col
    
    def make_col_prompt(self, col: str, example_values: Iterable, num_example_values: int = 5) -> str:
        """
        Based on the column title and some possible values of that columns, try to make a prompt
        """
        col_prompt = f"{col}: "
        for i in range(min(num_example_values, len(example_values))):
            col_prompt += f"{example_values[i]}, "
        return col_prompt

    def create_col_embeddings_from_model(self, col: str | List[str]) -> np.ndarray | torch.Tensor:
        """
        Create embeddings from string represent col name or a prompt of that col
        """
        input = self.embeddings_model_tokenizer(col, padding=True, truncation=True, return_tensors='pt')

        # get token embeddings
        with torch.no_grad():
            output = self.embeddings_model(**input)
        token_embeddings = output[0]

        # extract mask and mean pooling for sentence embeddings
        input_mask_expanded = input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
        col_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

        # final normalization
        col_embeddings = F.normalize(col_embeddings, p=2, dim=1)

        return col_embeddings
    
    def reranking_with_llm(self, col: str, candidates: List[str]) -> str:
        """
        LLM acts as a re-ranker to pick the best match from a list of candidates.
        """
        if not self.use_llm:
            raise Exception("LLM use has not been enabled in the model, please re-init with use_llm=True")
        
        # Format the candidates as a numbered list for the LLM
        candidates_str = "\n".join([f"\t{inx+1}. {c}" for inx, c in enumerate(candidates)])
        prompt = f"""Task: Map clinical table headers to GWAS standard ontology, return a single number for the best choice

Header: "p-value: 0.001, 5e-8, 0.43"
Candidates: 
    1. P-value: The statistical significance of the association. Keywords: P, P-value, P_adj, FDR. Examples: 5.0E-08, 0.0012, 1.2 x 10^-5, 0.05.
    2. Effect Size: The magnitude and direction of the association. Keywords: Beta, OR, HR, Estimate. Examples: Beta=0.25, OR=1.45, HR=1.12, Log(OR)=0.37.
    3. SNP: Variant identifier, or snp idenifier, or chr:pos. Keywords: chr:position, chr:pos, Variant, rsID, RS number, MarkerName, rs. Examples: rs12345, 20:45269867, 19:45411941:T:C, chr19:45411941, rs429358 (APOE ε4).
Best Match: 1

Header: "rs_number: rs123, rs456, rs789"
Candidates: 
    1. Chr: Genomic chromosome identifier. Keywords: CHR, Chrom, Chromosome. Examples: 1, 19, X, chr19, chrX.
    2. Position: Genomic coordinate location. Keywords: BP, POS, Base Pair, start, end. Examples: 45411941, 10240500:10248600 (range), build 37.
    3. SNP: Variant identifier, or snp idenifier, or chr:pos. Keywords: chr:position, chr:pos, Variant, rsID, RS number, MarkerName, rs. Examples: rs12345, 20:45269867, 19:45411941:T:C, chr19:45411941, rs429358 (APOE ε4).
Best Match: 2

Header: "{col}"
Candidates: 
{candidates_str}
Best Match: """

        allowed_indices = [str(i+1) for i in range(len(candidates))]
        allowed_token_ids = [self.llm_model_tokenizer.encode(idx, add_special_tokens=False)[0] for idx in allowed_indices]
        
        # logit bias to limit tokens that can be output
        bias_processor = SingleTokenBiasProcessor(allowed_token_ids, 100.0)
        logits_processor = LogitsProcessorList([bias_processor])

        # 4. Generate exactly ONE token
        inputs = self.llm_model_tokenizer(prompt, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            output = self.llm_model.generate(
                **inputs,
                max_new_tokens=1,      # Force exactly one token
                logits_processor=logits_processor, # Force it to be one of our numbers
                pad_token_id=self.llm_model_tokenizer.eos_token_id,
                do_sample=False        # Greedy decoding for consistency
            )

        # 5. Extract and Convert to Integer
        new_token = output[0][-1]
        predicted_text = self.llm_model_tokenizer.decode(new_token).strip()
        
        try:
            idx = int(predicted_text) - 1 # Convert back to 0-based list index
            return candidates[idx]
        except (ValueError, IndexError):
            return col
        
    def identify_main_role_in_multi_index_col(self, col: str) -> str:
        sub_col = col.split(".")
        candidates = []
        for i in range(len(sub_col)):
            main_role = f"{sub_col[i]} of "
            context = " and ".join([sub_col[j] for j in range(len(sub_col)) if j != i])
            main_role_and_context = main_role + context
            candidates.append(main_role_and_context)
        # make candidate str
        candidates_str = "\n".join([f"\t{inx+1}. {c}" for inx, c in enumerate(candidates)])
        prompt = f"""Task: Figure out which is the real meaning of the column in gwas table, return a number for the best choice

Candidates: 
    1. Multi-ethnic meta-analysis of Effect
    2. Effect of Multi-ethnic meta-analysis
Best Match: 2

Candidates: 
    1. p -value (10,000 permutations) of APOE *4 + females
    2. APOE *4 + females of p -value (10,000 permutations) 
Best Match: 1

Candidates:
    1. GWASmeta-analysis or chr:position
    2. chr:position or GWASmeta-analysis
Best Match: 2

Candidates: 
{candidates_str}
Best Match: """
        
        allowed_indices = [str(i+1) for i in range(len(candidates))]
        allowed_token_ids = [self.llm_model_tokenizer.encode(idx, add_special_tokens=False)[0] for idx in allowed_indices]
        
        # logit bias to limit tokens that can be output
        bias_processor = SingleTokenBiasProcessor(allowed_token_ids, 100.0)
        logits_processor = LogitsProcessorList([bias_processor])

        # 4. Generate exactly ONE token
        inputs = self.llm_model_tokenizer(prompt, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            output = self.llm_model.generate(
                **inputs,
                max_new_tokens=1,      # Force exactly one token
                logits_processor=logits_processor, # Force it to be one of our numbers
                pad_token_id=self.llm_model_tokenizer.eos_token_id,
                do_sample=False        # Greedy decoding for consistency
            )

        # 5. Extract and Convert to Integer
        new_token = output[0][-1]
        predicted_text = self.llm_model_tokenizer.decode(new_token).strip()
        
        try:
            idx = int(predicted_text) - 1 # Convert back to 0-based list index
            return sub_col[idx]
        except (ValueError, IndexError):
            return col
        
    def match_single_col_to_ref_col(self, col: str) -> Tuple[str, float]:
        """
        match a single column to reference col given column, the list of referencing col and their embeddings
        """
        # calculate column embedding
        # col_embeddings = embeddings_model.encode(col, normalize_embeddings=True)
        # multi_index_pattern = r".+\..+"
        # if re.search(multi_index_pattern, col):
        #     col = col.replace(".", " ")
        #     # sub_col_lst = col.split(".")
        #     # sub_col_embeddings = create_embeddings_from_model(sub_col_lst, embeddings_model, embeddings_model_tokenizer)
        #     # col_embeddings = torch.mean(sub_col_embeddings, dim = 0)
        #     col = col.replace(".", " ")
        #     col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
        # else:
        #     col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
        col_embeddings = self.create_col_embeddings_from_model(col)
        col_embeddings = col_embeddings.reshape(-1, 1)

        # calculate similarity score
        scores = torch.matmul(self.referencing_col_embeddings, col_embeddings).reshape(-1)

        # sort similairty score
        top_k_indices = torch.argsort(scores, descending=True)

        # verify if we even got good enough similarity
        best_inx = top_k_indices[0].item()
        best_score = scores[best_inx].item()
        # second_best_inx = top_k_indices[1].item()
        # second_best_score = scores[second_best_inx].item()
        # need a threshold for score or else, just return col
        # if best_score < 0.4: 
        #     return (col, 1)
        
        # # now do rerank
        # candidates = [ref_col_lst[inx] for inx in top_k_indices if scores[inx] >= 0.4]
        # candidates_scores = [scores[inx] for inx in top_k_indices if scores[inx] >= 0.4]
        # best_ref_col, best_ref_col_scores = reranking_from_model(col, candidates, candidates_scores)
        # return (best_ref_col, best_ref_col_scores)

        if best_score >= 0.4:
            return (self.referencing_col_lst[best_inx], best_score)
        return (col, 1)

    def match_many_col_to_ref_col(self, df: pd.DataFrame) -> Dict:
        """
        Given a list of column and a dataframe (could be dict later on if that fits better),
        try to match each column to the best fitted reference col, 
        return a dict of ref col : list of (col, cleaned col prompt, score)
        """

        # conduct matching
        multi_index_pattern = r"^.+\..+$" # need multi index pattern for handling multi index
        ref_col_to_col_lst = {}
        for col in df.columns:

            # extract values needed for prompt
            cleaned_col = self.clean_col(col)
            example_values = df[col].unique().tolist()
            # prompt: {col}: example, need to delete all : first

            if re.search(multi_index_pattern, cleaned_col.strip()):
                # try to assess each part and see if which one have highest score
                best_ref_col, best_score = None, 0
                best_cleaned_sub_col_prompt = None
                for sub_col in cleaned_col.split("."):

                    # make column prompt for each sub col
                    cleaned_sub_col_prompt = self.make_col_prompt(sub_col, example_values)
                    
                    # matching and compare
                    ref_col, score = self.match_single_col_to_ref_col(cleaned_sub_col_prompt)
                    if score > best_score and ref_col != cleaned_sub_col_prompt:
                        best_ref_col = ref_col
                        best_score = score
                        best_cleaned_sub_col_prompt = cleaned_sub_col_prompt
                
                # if we have a best one vs not
                if best_ref_col is not None:
                    if best_ref_col not in ref_col_to_col_lst:
                        ref_col_to_col_lst[best_ref_col] = []
                    ref_col_to_col_lst[best_ref_col].append((col, best_cleaned_sub_col_prompt, score))
                else:
                    if col not in ref_col_to_col_lst:
                        ref_col_to_col_lst[col] = []
                    ref_col_to_col_lst[col].append((col, best_cleaned_sub_col_prompt, 1))
            else:
                # match a single col with best fit referencing col and return a dict
                # make the col prompt
                # extra steps to remove .
                cleaned_col = cleaned_col.replace(".", "")
                cleaned_col_prompt = self.make_col_prompt(cleaned_col, example_values)

                # matching for best col
                ref_col, score = self.match_single_col_to_ref_col(cleaned_col_prompt)
                # if we still get same col
                if ref_col == cleaned_col_prompt: 
                    ref_col = col
                if ref_col not in ref_col_to_col_lst:
                    ref_col_to_col_lst[ref_col] = []
                ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))
            # ref_col, score = match_single_col_to_ref_col(cleaned_col_prompt, ref_col_lst, ref_col_embeddings)
            # if ref_col == cleaned_col_prompt: 
            #     ref_col = col
            # if ref_col not in ref_col_to_col_lst:
            #     ref_col_to_col_lst[ref_col] = []
            # ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))

        return ref_col_to_col_lst
    
    def match_single_col_to_ref_col_with_llm(self, col: str, num_candidates: int = 3) -> str:
        """
        match a single column to reference col given column, the list of referencing col and their embeddings
        """
        if not self.use_llm:
            raise Exception("LLM use has not been enabled in the model, please re-init with use_llm=True")
        
        # calculate column embedding
        # col_embeddings = embeddings_model.encode(col, normalize_embeddings=True)
        # multi_index_pattern = r".+\..+"
        # if re.search(multi_index_pattern, col):
        #     col = col.replace(".", " ")
        #     # sub_col_lst = col.split(".")
        #     # sub_col_embeddings = create_embeddings_from_model(sub_col_lst, embeddings_model, embeddings_model_tokenizer)
        #     # col_embeddings = torch.mean(sub_col_embeddings, dim = 0)
        #     col = col.replace(".", " ")
        #     col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
        # else:
        #     col_embeddings = create_embeddings_from_model(col, embeddings_model, embeddings_model_tokenizer)
        col_embeddings = self.create_col_embeddings_from_model(col)
        col_embeddings = col_embeddings.reshape(-1, 1)

        # calculate similarity score
        scores = torch.matmul(self.referencing_col_embeddings, col_embeddings).reshape(-1)

        # sort similairty score
        top_k_indices = torch.argsort(scores, descending=True)

        # verify if we even got good enough similarity
        best_inx = top_k_indices[0].item()
        best_score = scores[best_inx].item()
        # second_best_inx = top_k_indices[1].item()
        # second_best_score = scores[second_best_inx].item()
        # need a threshold for score or else, just return col
        # if best_score < 0.4: 
        #     return (col, 1)
        
        # # now do rerank
        # candidates = [ref_col_lst[inx] for inx in top_k_indices if scores[inx] >= 0.4]
        # candidates_scores = [scores[inx] for inx in top_k_indices if scores[inx] >= 0.4]
        # best_ref_col, best_ref_col_scores = reranking_from_model(col, candidates, candidates_scores)
        # return (best_ref_col, best_ref_col_scores)

        if best_score < 0.4:
            return col
        else:
            # extract top 3 candidates
            candidates = []
            for i in range(num_candidates):
                inx = top_k_indices[i]
                # need to add the column with context, not just column
                candidates.append(self.referencing_col_context_lst[inx])
            best_col = self.reranking_with_llm(col, candidates)
            # after this, map back to normal column
            if best_col in self.referencing_col_context_to_col:
                best_col = self.referencing_col_context_to_col[best_col]
            return best_col

    def match_many_col_to_ref_col_with_llm(self, df: pd.DataFrame) -> Dict:
        """
        Given a list of column and a dataframe (could be dict later on if that fits better),
        try to match each column to the best fitted reference col, 
        return a dict of ref col : list of (col, cleaned col prompt, score)
        """
        if not self.use_llm:
            raise Exception("LLM use has not been enabled in the model, please re-init with use_llm=True")
        
        # prepare the embeddings for reference col since we reuse them
        # conduct matching
        multi_index_pattern = r"^.+\..+$" # need multi index pattern for handling multi index
        ref_col_to_col_lst = {}
        for col in df.columns:

            # extract values needed for prompt
            cleaned_col = self.clean_col(col)
            example_values = df[col].unique().tolist()
            # prompt: {col}: example, need to delete all : first

            # if re.search(multi_index_pattern, cleaned_col.strip()):
            #     # try to assess each part and see if which one have highest score
            #     best_candidates = []
            #     for sub_col in cleaned_col.split("."):

            #         # make column prompt for each sub col
            #         cleaned_sub_col_prompt = self.make_col_prompt(sub_col, example_values)
                    
            #         # matching and compare
            #         ref_col = self.match_single_col_to_ref_col_with_llm(cleaned_sub_col_prompt)
            #         # need to convert because we do the reranking again later, we must convert to col with context
            #         if ref_col in self.referencing_col_to_col_context:
            #             best_candidates.append(self.referencing_col_to_col_context[ref_col])
            #         else:
            #             best_candidates.append(ref_col)
            #     # if we have a best one vs not
            #     if len(best_candidates) > 0:
            #         cleaned_col = cleaned_col.replace(".", " ")
            #         cleaned_col_prompt = self.make_col_prompt(cleaned_col, example_values)
            #         best_ref_col = self.reranking_with_llm(cleaned_col_prompt, best_candidates)

            #         # after getting best ref col, since it is still col with context, need converting back
            #         if best_ref_col in self.referencing_col_context_to_col:
            #             best_ref_col = self.referencing_col_context_to_col[best_ref_col]

            #         if best_ref_col not in ref_col_to_col_lst:
            #             ref_col_to_col_lst[best_ref_col] = []
            #         ref_col_to_col_lst[best_ref_col].append((col, cleaned_col_prompt))
            #     else:
            #         if col not in ref_col_to_col_lst:
            #             ref_col_to_col_lst[col] = []
            #         ref_col_to_col_lst[col].append((col, None))
            # else:
            #     # match a single col with best fit referencing col and return a dict
            #     # make the col prompt
            #     # extra steps to remove .
            #     cleaned_col = cleaned_col.replace(".", "")
            #     cleaned_col_prompt = self.make_col_prompt(cleaned_col, example_values)

            #     # matching for best col
            #     ref_col = self.match_single_col_to_ref_col_with_llm(cleaned_col_prompt)
            #     # if we still get same col
            #     if ref_col == cleaned_col_prompt: 
            #         ref_col = col
            #     if ref_col not in ref_col_to_col_lst:
            #         ref_col_to_col_lst[ref_col] = []
            #     ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt))
            # ref_col, score = match_single_col_to_ref_col(cleaned_col_prompt, ref_col_lst, ref_col_embeddings)
            # if ref_col == cleaned_col_prompt: 
            #     ref_col = col
            # if ref_col not in ref_col_to_col_lst:
            #     ref_col_to_col_lst[ref_col] = []
            # ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))

            if re.search(multi_index_pattern, cleaned_col.strip()):
                main_role = self.identify_main_role_in_multi_index_col(cleaned_col)
                cleaned_col_prompt = self.make_col_prompt(main_role, example_values)
            else:
                cleaned_col = cleaned_col.replace(".", "")
                cleaned_col_prompt = self.make_col_prompt(cleaned_col, example_values)
            ref_col, score = self.match_single_col_to_ref_col(cleaned_col_prompt)
            # if we still get same col
            if ref_col == cleaned_col_prompt: 
                ref_col = col
            if ref_col not in ref_col_to_col_lst:
                ref_col_to_col_lst[ref_col] = []
            ref_col_to_col_lst[ref_col].append((col, cleaned_col_prompt, score))

        return ref_col_to_col_lst

In [ ]:
gwas_col_matching_engine = GWASColumnMatchingEngine(referencing_cols, use_llm = True, device = device)

for file in os.listdir("./tables"):
    if "table" in file and ".csv" in file and "harmonized" not in file:
        print(file)
        df = pd.read_csv(f"./tables/{file}")
        df.columns = ['' if 'Unnamed:' in col else col for col in df.columns]
        col_to_ref_col = gwas_col_matching_engine.match_many_col_to_ref_col(df)
        for ref_col in col_to_ref_col:
            print(f"{ref_col}: {col_to_ref_col[ref_col]}")
        print()

Final formatting of the tables

In [ ]:
# We now can try to use these dictionary to make a final dataset
# need a list of columns that we sure that there might be multiple copies of it that we can melt them into many rows
possible_ref_col_to_melt = ["P-value", "Effect Size", "AF"]
gwas_col_matching_engine = GWASColumnMatchingEngine(referencing_cols)
def format_original_table(df, remove_unique_col = False):
    # map the columns
    new_col_to_old_col_lst = gwas_col_matching_engine.match_many_col_to_ref_col(df)
    df_with_ref_col = None 
    new_col_to_not_melt = [] # list of columns that are stable and not need to be melt
    new_col_to_old_col_lst_to_melt = {}
    for new_col in new_col_to_old_col_lst:
        if len(new_col_to_old_col_lst[new_col]) == 1:
            if (not remove_unique_col) or (remove_unique_col and new_col_to_old_col_lst[new_col][0][2] != 1):
                if df_with_ref_col is None:
                    df_with_ref_col = df[[new_col_to_old_col_lst[new_col][0][0]]]
                    df_with_ref_col = df_with_ref_col.rename({new_col_to_old_col_lst[new_col][0][0]: new_col}, axis = 1)
                else:
                    df_with_ref_col[new_col] = df[new_col_to_old_col_lst[new_col][0][0]]
                # add these single col to the list of not melt
                new_col_to_not_melt.append(new_col)
        else:
            if new_col in possible_ref_col_to_melt:
                old_col_lst = []
                for col, _, _ in new_col_to_old_col_lst[new_col]:
                    old_col_lst.append(col)
                    if df_with_ref_col is None:
                        df_with_ref_col = df[[col]]
                    else:
                        df_with_ref_col[col] = df[col]
                new_col_to_old_col_lst_to_melt[new_col] = old_col_lst.copy()
            else:
                # make multiple copies with notes
                for inx, (col, _, _) in enumerate(new_col_to_old_col_lst[new_col]):
                    if df_with_ref_col is None:
                        df_with_ref_col = df[[col]]
                        df_with_ref_col = df_with_ref_col.rename({col: f"{new_col}_{inx + 1}"}, axis = 1)
                    else:
                        df_with_ref_col[f"{new_col}_{inx + 1}"] = df[col]
                    df_with_ref_col[f"{new_col}_{inx + 1} notes"] = col
                    # add these cols in group but not need to melt
                    new_col_to_not_melt.append(f"{new_col}_{inx + 1}")
                    new_col_to_not_melt.append(f"{new_col}_{inx + 1} notes")
    # Melting stage
    if len(new_col_to_old_col_lst_to_melt) > 0:
        # now melting column in same groups
        # Instead of keep melting, for each group, we make a new dataset of 
        # [stable col] + [to be melt col] => melt them as a new df
        # do this for each gorup and then join together based on stable col
        df_with_melt_col = None 
        for new_col in new_col_to_old_col_lst_to_melt:
            temp_df = deepcopy(df_with_ref_col[new_col_to_not_melt + new_col_to_old_col_lst_to_melt[new_col]])
            # create a temp row id for stable join
            temp_df["_row_id"] = np.arange(temp_df.shape[0])
            temp_df = temp_df.melt(
                id_vars = new_col_to_not_melt + ["_row_id"],    
                value_vars = new_col_to_old_col_lst_to_melt[new_col], 
                var_name = f"{new_col} notes",      
                value_name = f"{new_col}"
            )
            if df_with_melt_col is None:
                df_with_melt_col = deepcopy(temp_df)
            else:
                df_with_melt_col = df_with_melt_col.merge(temp_df, how = "inner", on = ["_row_id"] + new_col_to_not_melt)
        df_with_melt_col = df_with_melt_col.drop("_row_id", axis = 1)
        return df_with_melt_col
    else:
        return df_with_ref_col

In [ ]:
modified_df_all = None
for file in os.listdir("./tables"):
    if ".csv" in file and "table" in file and "harmonized" not in file:
        df = pd.read_csv(f"./tables/{file}")
        df.columns = ['' if 'Unnamed:' in col else col for col in df.columns]
        modified_df = format_original_table(df, remove_unique_col = True)
        modified_df["file_name"] = file
        if modified_df_all is None:
            modified_df_all = modified_df.copy()
        else:
            modified_df_all = pd.concat([modified_df_all, modified_df], ignore_index = True)
        modified_df.to_csv(f"./harmonized_tables/{file.replace('.csv', '')}_harmonized.csv", index = False)
modified_df_all.to_csv("./harmonized_tables/harmonized_table.csv", index = False)

Testing for table harmonization from table_link_to_excel code

In [2]:
# first we need to extract a set of id from table
def extract_table_id_lst_from_pmc(pmcid):
    url = f"https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/{pmcid}/unicode"

    response = requests.get(url)
    tables_id_set = set()
    if response.status_code == 200:
        data = response.json()
        for d in data:
            doc = d["documents"]
            for p in doc:
                passage = p["passages"]
                for item in passage:
                    if "table" in item.get("infons", "").get("type", "").lower() and "id" in item.get("infons", ""):
                        tables_id_set.add(item.get("infons", "").get("id", ""))
                        
        print(f"Successfully retrieve list of tables' id")
    else:
        print(f"Failed to retrieve list of tables' id {response.status_code}")
        
    return [id for id in list(tables_id_set) if id != ""]

In [3]:
def contains_valid_snp(row):
    row_snp_pattern = r"\b(?:rs|s)\d+\S*"
    return any(re.search(row_snp_pattern, str(value)) for value in row)

def contains_valid_pvalue(row):
    row_pvalue_pattern = r"\d+\.\d+"
    return any(re.search(row_pvalue_pattern, str(value)) for value in row)

In [4]:
def table_link_to_excel(pmid, pmcid):
    # extract list of table id directly
    table_id_list = extract_table_id_lst_from_pmc(pmcid) 
    # print(table_id_list)

    # possible_table_id = ["T", "Tab", "Table", "Tbl", "t", "tab", "table", "tbl"]

    # for i in range(1, num_tables + 1):
    for table_id in table_id_list:
        table_name = f'tables/{pmid}_{pmcid}_{table_id}_from_pmc.xlsx'
        found_table = False
        # for pti in possible_table_id:
        try:
            if BeautifulSoup is None:
                raise RuntimeError("beautifulsoup4 is not installed. Run: python3 -m pip install beautifulsoup4")

            # pmcid, url_table_id = _extract_pmc_info(f"https://pmc.ncbi.nlm.nih.gov/articles/{pmcid}")
            # resolved_table_id = table_id

            # Fast-path fallback for PMC direct table assets (often bypasses page-level 403).
            if pmcid and table_id:
                direct_tables = _try_pmc_direct_table_download(pmcid, table_id)
                if direct_tables:
                    with pd.ExcelWriter(table_name, engine="openpyxl") as writer:
                        for i, df in enumerate(direct_tables, start=1):
                            df2 = _flatten_columns(df)
                            df2.to_excel(writer, index=False, sheet_name=sanitize_sheet_name("", f"table_{i}"))
                            found_table = True
                
            if not found_table:
                html = ""
                soup = None
                parse_with_xml = False
                try:
                    html = fetch_html(f"https://pmc.ncbi.nlm.nih.gov/articles/{pmcid}")
                    soup = BeautifulSoup(html, "lxml")
                except Exception as e:
                    # PMC pages may return 403 to scripts; fallback to Europe PMC XML.
                    if pmcid:
                        xml = fetch_pmc_fulltext_xml(pmcid)
                        soup = BeautifulSoup(xml, "xml")
                        parse_with_xml = True
                    else:
                        raise e

                if table_id:
                    tables = pick_table(soup, table_id=table_id, table_selector=None, table_index=0)
                # elif args.all_tables:
                #     if parse_with_xml:
                #         wraps = soup.find_all("table-wrap")
                #         tables = [w.find("table") for w in wraps if w.find("table") is not None]
                #     else:
                #         tables = soup.select(args.table_selector) if args.table_selector else soup.find_all("table")
                #     if not tables:
                #         raise ValueError("No matched tables found.")
                # else:
                #     tables = pick_table(
                #         soup,
                #         table_id=None,
                #         table_selector=args.table_selector,
                #         table_index=args.table_index,
                #     )
                with pd.ExcelWriter(table_name, engine="openpyxl") as writer:
                    for i, table in enumerate(tables, start=1):
                        df = table_to_dataframe(table)
                        caption = ""
                        cap = table.find("caption")
                        if cap:
                            caption = _clean_text(cap.get_text(" ", strip=True))
                        sheet = sanitize_sheet_name(caption, fallback=f"table_{i}")
                        df2 = _flatten_columns(df)
                        df2.to_excel(writer, index=False, header=False, sheet_name=sheet)
                        found_table = True
        
        except:
            continue
            
        if found_table:
            # filter table
            df = pd.read_excel(table_name)
            df["valid_row"] = df.apply(lambda x: contains_valid_snp(x) and contains_valid_pvalue(x), axis=1)
            df = df[df["valid_row"]].drop("valid_row", axis=1).reset_index().drop("index", axis = 1)
            if df.shape[0] > 0:
                with pd.ExcelWriter(table_name, engine="openpyxl") as writer:
                    df.to_excel(writer, index=False)

        else:
            # delete file
            os.remove(table_name)

In [5]:
# Now try to map the columns with the actual col in advp
referencing_col_df = pd.read_csv("Rules for harmonizing ADVP papers - Main cols.csv")
gwas_column_matching_engine = GWASColumnMatchingEngine(referencing_col_df)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
test_papers_info = [
    (30448613, "PMC6331247"), (30979435, "PMC6783343"), (31055733, "PMC6544706"),  (30617256, "PMC6836675"),
    (30820047, "PMC6463297"), (29458411, "PMC5819208"), (29777097, "PMC5959890"), (30651383, "PMC6369905"),
    (31497858, "PMC6736148"), (30930738, "PMC6425305"), (31426376, "PMC6723529"), (29967939, "PMC6280657"),
    (29107063, "PMC5920782"), (29274321, "PMC5938137"), (30413934, "PMC6358498"), (30805717, "PMC7193309"),
    (30636644, "PMC6330399"), (29752348, "PMC5976227"), (28560309, "PMC5440281"), (27899424, "PMC5237405"),
]

referencing_col_df = pd.read_csv("Rules for harmonizing ADVP papers - Main cols.csv")
gwas_column_matching_engine = GWASColumnMatchingEngine(referencing_col_df)

file_name_to_matching = {}
for pmid, pmcid in test_papers_info:
    try:
        table_link_to_excel(pmid, pmcid)
        print(f"Success in extracting from {pmid}-{pmcid}")
    except Exception as e:
        print(f"Error in extracting from {pmid}-{pmcid} with error {e}")

    try:
        harmonized_df_all = pd.DataFrame(columns = referencing_col_df["column"].to_list())
        for file_name in os.listdir("tables"):
            if str(pmid) in file_name and pmcid in file_name:
                df = pd.read_excel(f"tables/{file_name}")

                # save matching dict for debug
                file_name_to_matching[file_name] = gwas_column_matching_engine.match_many_col_to_ref_col(df)

                harmonized_df = format_original_table(df, gwas_column_matching_engine, remove_unique_col = True)
                if harmonized_df_all is None:
                    harmonized_df_all = harmonized_df.copy()
                else:
                    harmonized_df_all = pd.concat([harmonized_df_all, harmonized_df], ignore_index = True)
        for col in referencing_col_df["column"].unique():
            if col not in harmonized_df_all.columns:
                harmonized_df_all[col] = pd.NA
        harmonized_df_all.to_csv(f"pred_tables/{pmid}_{pmcid}.csv", index = False)
        print(f"Success in harmonizing from {pmid}-{pmcid}")
    except Exception as e:
        print(f"Error in harmonizing from {pmid}-{pmcid} with error {e}")
with open("test_matching_dict.json", "w") as f:
    json.dump(file_name_to_matching, f, indent=4)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Successfully retrieve list of tables' id
Success in extracting from 30448613-PMC6331247
Success in harmonizing from 30448613-PMC6331247
Successfully retrieve list of tables' id
Success in extracting from 30979435-PMC6783343


/var/folders/5b/jn4p_4vx7j9gx2jdrmbjlxnr0000gn/T/ipykernel_5345/2412568355.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  harmonized_df_all = pd.concat([harmonized_df_all, harmonized_df], ignore_index = True)


Success in harmonizing from 30979435-PMC6783343
Error in extracting from 31055733-PMC6544706 with error Expecting value: line 1 column 2 (char 1)
Success in harmonizing from 31055733-PMC6544706
Successfully retrieve list of tables' id
Success in extracting from 30617256-PMC6836675


/var/folders/5b/jn4p_4vx7j9gx2jdrmbjlxnr0000gn/T/ipykernel_5345/2412568355.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  harmonized_df_all = pd.concat([harmonized_df_all, harmonized_df], ignore_index = True)


...

Success in harmonizing from 30805717-PMC7193309
Successfully retrieve list of tables' id
Success in extracting from 30636644-PMC6330399


/var/folders/5b/jn4p_4vx7j9gx2jdrmbjlxnr0000gn/T/ipykernel_5345/2412568355.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  harmonized_df_all = pd.concat([harmonized_df_all, harmonized_df], ignore_index = True)


Success in harmonizing from 30636644-PMC6330399
Successfully retrieve list of tables' id
Success in extracting from 29752348-PMC5976227
Success in harmonizing from 29752348-PMC5976227
Successfully retrieve list of tables' id
Success in extracting from 28560309-PMC5440281


/var/folders/5b/jn4p_4vx7j9gx2jdrmbjlxnr0000gn/T/ipykernel_5345/2412568355.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  harmonized_df_all = pd.concat([harmonized_df_all, harmonized_df], ignore_index = True)


Success in harmonizing from 28560309-PMC5440281
Successfully retrieve list of tables' id
Success in extracting from 27899424-PMC5237405
Success in harmonizing from 27899424-PMC5237405
